In [2]:
!wget -O arch.pgn.zst https://database.lichess.org/standard/lichess_db_standard_rated_2015-09.pgn.zst

--2024-10-09 18:33:07--  https://database.lichess.org/standard/lichess_db_standard_rated_2015-09.pgn.zst
Resolving database.lichess.org (database.lichess.org)... 141.95.66.62, 2001:41d0:700:5e3e::
Connecting to database.lichess.org (database.lichess.org)|141.95.66.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 547459020 (522M) [application/octet-stream]
Saving to: ‘arch.pgn.zst’

arch.pgn.zst        100%[===================>] 522.10M  25.8MB/s    in 21s     

2024-10-09 18:33:29 (24.8 MB/s) - ‘arch.pgn.zst’ saved [547459020/547459020]



In [3]:
!apt-get install zstd

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 1s (594 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 119626 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...


In [4]:
!unzstd arch.pgn.zst

arch.pgn.zst        : 2903022236 bytes 


In [5]:
!wget https://github.com/duusha/chess_engine/raw/main/auxiliary_func.py
!wget https://github.com/duusha/chess_engine/raw/main/dataset.py
!wget https://github.com/duusha/chess_engine/raw/main/model.py



--2024-10-09 18:33:45--  https://github.com/duusha/chess_engine/raw/main/auxiliary_func.py
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/duusha/chess_engine/main/auxiliary_func.py [following]
--2024-10-09 18:33:45--  https://raw.githubusercontent.com/duusha/chess_engine/main/auxiliary_func.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1443 (1.4K) [text/plain]
Saving to: ‘auxiliary_func.py’

auxiliary_func.py   100%[===================>]   1.41K  --.-KB/s    in 0s      

2024-10-09 18:33:45 (32.5 MB/s) - ‘auxiliary_func.py’ saved [1443/1443]

--2024-10-09 18:33:46--  https://github.com/duu

In [6]:
!pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148499 sha256=c50d4f7fa345e5832ab51df80fb47bd1771095ffc61ff17dd487913219294ab1
  Stored in directory: /root/.cache/pip/wheels/2e/2d/23/1bfc95db984ed3ecbf6764167dc7526d0ab521cf9a9852544e
Successfully built chess


In [7]:
import os
import numpy as np # type: ignore
import time
import torch
import torch.nn as nn # type: ignore
import torch.optim as optim # type: ignore
from torch.utils.data import DataLoader # type: ignore
from chess import pgn # type: ignore
from tqdm import tqdm # type: ignore


In [8]:
# def load_pgn(file_path):
#     games = []
#     with open(file_path, 'r') as pgn_file:
#         while True:
#             game = pgn.read_game(pgn_file)
#             if game is None:
#                 break
#             games.append(game)
#     return games

# files = [file for file in os.listdir("Lichess Elite Database") if file.endswith(".pgn")]
# LIMIT_OF_FILES = min(len(files), 28)
# games = []
# i = 1
# for file in tqdm(files):
#     games.extend(load_pgn(f"Lichess Elite Database/{file}"))
#     if i >= LIMIT_OF_FILES:
#         break
#     i += 1
with open("arch.pgn", 'r') as pgn_file:
    games = []
    count = 0
    start = time.time()
    while True:
        # start = time.time()
        game = pgn.read_game(pgn_file)
        print(f"\rCount games: {count} Time taken to read: {time.time() - start}s", end = '', flush=True)
        if game is None or count > 10000:
            break
        games.append(game)
        count += 1

Count games: 10001 Time taken to read: 40.95614314079285s

In [9]:
games = games[:-1]
print(f"GAMES PARSED: {len(games)}")

GAMES PARSED: 10000


In [10]:
from auxiliary_func import create_input_for_nn, encode_moves
### add train test split

X, y = create_input_for_nn(games)

print(f"NUMBER OF SAMPLES: {len(y)}")

y, move_to_int = encode_moves(y)
num_classes = len(move_to_int)

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

100%|██████████| 10000/10000 [01:21<00:00, 122.89it/s]


NUMBER OF SAMPLES: 654122


In [11]:
from dataset import ChessDataset
from model import ChessModel

# Create Dataset and DataLoader
dataset = ChessDataset(X, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Model Initialization
model = ChessModel(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Using device: cpu


In [ ]:
# num_epochs = 50
# for epoch in range(num_epochs):
#     start_time = time.time()
#     model.train()
#     running_loss = 0.0
#     for inputs, labels in tqdm(dataloader):
#         inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
#         optimizer.zero_grad()

#         outputs = model(inputs)  # Raw logits

#         # Compute loss
#         loss = criterion(outputs, labels)
#         loss.backward()

#         # Gradient clipping
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

#         optimizer.step()
#         running_loss += loss.item()
#     end_time = time.time()
#     epoch_time = end_time - start_time
#     minutes: int = int(epoch_time // 60)
#     seconds: int = int(epoch_time) - minutes * 60
#     print(f'Epoch {epoch + 1 + 50}/{num_epochs + 1 + 50}, Loss: {running_loss / len(dataloader):.4f}, Time: {minutes}m{seconds}s')


num_epochs = 50
start_epoch = 0  # Can be changed if resuming training from a specific epoch
losses = []  # To store losses for plotting

for epoch in range(start_epoch, start_epoch + num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0

    # Track correct predictions for accuracy calculation
    correct = 0
    total = 0

    for inputs, labels in tqdm(dataloader, desc=f'Epoch {epoch + 1}/{start_epoch + num_epochs}'):
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()

        outputs = model(inputs)  # Raw logits

        # Compute loss
        loss = criterion(outputs, labels)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Calculate epoch loss and accuracy
    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = 100 * correct / total
    losses.append(epoch_loss)  # Store loss for plotting

    # Measure epoch time
    end_time = time.time()
    epoch_time = end_time - start_time
    minutes = int(epoch_time // 60)
    seconds = int(epoch_time % 60)

    # Print epoch statistics
    print(f'Epoch {epoch + 1}/{start_epoch + num_epochs}, '
          f'Loss: {epoch_loss:.4f}, '
          f'Accuracy: {epoch_accuracy:.2f}%, '
          f'Time: {minutes}m{seconds}s')

    # Optionally update the learning rate scheduler
    # scheduler.step() if using a scheduler

    ### TODO: add test part, check learning rate, add saving best.pt and last.pt


Epoch 1/50: 100%|██████████| 10221/10221 [02:18<00:00, 74.02it/s]


Epoch 1/50, Loss: 4.5410, Accuracy: 13.91%, Time: 2m18s


Epoch 2/50: 100%|██████████| 10221/10221 [02:18<00:00, 73.84it/s]


Epoch 2/50, Loss: 3.4113, Accuracy: 21.07%, Time: 2m18s


Epoch 3/50: 100%|██████████| 10221/10221 [02:19<00:00, 73.33it/s]


...

Epoch 12/50: 100%|██████████| 10221/10221 [02:23<00:00, 71.06it/s]


Epoch 12/50, Loss: 1.9674, Accuracy: 43.70%, Time: 2m23s


Epoch 13/50: 100%|██████████| 10221/10221 [02:25<00:00, 70.44it/s]


Epoch 13/50, Loss: 1.9022, Accuracy: 45.20%, Time: 2m25s


Epoch 14/50:  47%|████▋     | 4765/10221 [01:07<01:26, 63.03it/s]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.show()